In [ ]:
!sudo apt-get update && sudo apt-get install -y apt-transport-https
!curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://apt.kubernetes.io/ kubernetes-xenial main" | sudo tee -a /etc/apt/sources.list.d/kubernetes.list
!sudo apt-get update
!sudo apt-get install -y kubectl

In [ ]:
!curl https://raw.githubusercontent.com/kubernetes/helm/master/scripts/get | bash

In [ ]:
!sudo apt-get install gir1.2-glib-2.0

In [ ]:
!pip install --upgrade dask[complete] dask-kubernetes dask-gateway 

**RESTART YOUR KERNEL**

In [1]:
!az login >> /dev/null

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BYE8CS6ZW to authenticate.
Failed to authenticate '{'additional_properties': {'displayName': 'Chevron', 'tenantCategory': 'Home', 'countryCode': 'US', 'domains': ['pasadenarefining.com', 'chevronstores.com', 'caltexgo.com', 'caltex.com.vn', 'chevronpac.com', 'chevronlummusglobal.com', 'chevronuktr.com', 'xponentcard.com', 'chevronenergy.com', 'chevronlubricants.com', 'cvxarea52.com', 'deepstar.org', 'arteco-coolants.eu', 'chevronpartnershipprograms.com', 'chevronadvocacynetwork.com', 'anlngservices.com', 'catchlightenergy.com', 'sasolchevron.com', 'caltex.com', 'richmondrefinery.com', 'chevronapc.onmicrosoft.com', 'chevroneur.onmicrosoft.com', 'chevrontexaco.net', 'chevrontexaco.com', 'tengizchevroil.net', 'tengizchevroil.com', 'tco.tengizchevroil.net', 'chevron.mail.onmicrosoft.com', 'chevron.onmicrosoft.com', 'ct.chevrontexaco.net', 'chevron.net', 'chevron.com']}, 'id': '/tenants/fd799

In [2]:
!az account show

{
  "environmentName": "AzureCloud",
  "homeTenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "id": "6560575d-fa06-4e7d-95fb-f962e74efd7a",
  "isDefault": true,
  "managedByTenants": [
    {
      "tenantId": "2f4a9838-26b7-47ee-be60-ccc1fdec5953"
    }
  ],
  "name": "Data4ML PM",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "copeters@microsoft.com",
    "type": "user"
  }
}


In [3]:
import os
os.environ['RG']  = 'cody-dask-aks-rg'
os.environ['AKS'] = 'dask-aks'
os.environ['LOC'] = 'westus2'
os.environ['VM']  = 'STANDARD_DS12_V2'
os.environ['MIN'] = '3'
os.environ['MAX'] = '100'

In [4]:
!az group create -n $RG -l $LOC

{
  "id": "/subscriptions/6560575d-fa06-4e7d-95fb-f962e74efd7a/resourceGroups/cody-dask-aks-rg",
  "location": "westus2",
  "managedBy": null,
  "name": "cody-dask-aks-rg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


In [6]:
!az aks create -n $AKS -g $RG --node-vm-size $VM --enable-cluster-autoscaler --min-count $MIN --max-count $MAX

{- Finished ..principal creation[##################################]  100.0000%
  "aadProfile": null,
  "addonProfiles": null,
  "agentPoolProfiles": [
    {
      "availabilityZones": null,
      "count": 3,
      "enableAutoScaling": true,
      "enableNodePublicIp": null,
      "maxCount": 100,
      "maxPods": 110,
      "minCount": 3,
      "name": "nodepool1",
      "nodeLabels": null,
      "nodeTaints": null,
      "orchestratorVersion": "1.15.10",
      "osDiskSizeGb": 100,
      "osType": "Linux",
      "provisioningState": "Succeeded",
      "scaleSetEvictionPolicy": null,
      "scaleSetPriority": null,
      "tags": null,
      "type": "VirtualMachineScaleSets",
      "vmSize": "Standard_DS2_v2",
      "vnetSubnetId": null
    }
  ],
  "apiServerAccessProfile": null,
  "dnsPrefix": "dask-aks-cody-dask-aks-rg-656057",
  "enablePodSecurityPolicy": null,
  "enableRbac": true,
  "fqdn": "dask-aks-cody-dask-aks-rg-656057-a7bd71c0.hcp.westus2.azmk8s.io",
  "id": "/subscriptions/

In [7]:
!az aks show -n $AKS -g $RG

{
  "aadProfile": null,
  "addonProfiles": null,
  "agentPoolProfiles": [
    {
      "availabilityZones": null,
      "count": 3,
      "enableAutoScaling": true,
      "enableNodePublicIp": null,
      "maxCount": 100,
      "maxPods": 110,
      "minCount": 3,
      "name": "nodepool1",
      "nodeLabels": null,
      "nodeTaints": null,
      "orchestratorVersion": "1.15.10",
      "osDiskSizeGb": 100,
      "osType": "Linux",
      "provisioningState": "Succeeded",
      "scaleSetEvictionPolicy": null,
      "scaleSetPriority": null,
      "tags": null,
      "type": "VirtualMachineScaleSets",
      "vmSize": "Standard_DS2_v2"
    }
  ],
  "apiServerAccessProfile": null,
  "dnsPrefix": "dask-aks-cody-dask-aks-rg-656057",
  "enablePodSecurityPolicy": null,
  "enableRbac": true,
  "fqdn": "dask-aks-cody-dask-aks-rg-656057-a7bd71c0.hcp.westus2.azmk8s.io",
  "id": "/subscriptions/6560575d-fa06-4e7d-95fb-f962e74efd7a/resourcegroups/cody-dask-aks-rg/providers/Microsoft.ContainerService/

In [8]:
!az aks get-credentials -n $AKS -g $RG --overwrite-existing

Merged "dask-aks" as current context in /home/azureuser/.kube/config


In [9]:
!kubectl --namespace kube-system create serviceaccount tiller

serviceaccount/tiller created


In [10]:
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller

clusterrolebinding.rbac.authorization.k8s.io/tiller created


In [11]:
!helm init --service-account tiller --history-max 100 --wait

Creating /home/azureuser/.helm 
Creating /home/azureuser/.helm/repository 
Creating /home/azureuser/.helm/repository/cache 
Creating /home/azureuser/.helm/repository/local 
Creating /home/azureuser/.helm/plugins 
Creating /home/azureuser/.helm/starters 
Creating /home/azureuser/.helm/cache/archive 
Creating /home/azureuser/.helm/repository/repositories.yaml 
Adding stable repo with URL: https://kubernetes-charts.storage.googleapis.com 
Adding local repo with URL: http://127.0.0.1:8879/charts 
$HELM_HOME has been configured at /home/azureuser/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation


In [12]:
!kubectl patch deployment tiller-deploy --namespace=kube-system --type=json --patch='[{"op": "add", "path": "/spec/template/spec/containers/0/command", "value": ["/tiller", "--listen=localhost:44134"]}]'

deployment.extensions/tiller-deploy patched


In [13]:
!helm version

Client: &version.Version{SemVer:"v2.16.3", GitCommit:"1ee0254c86d4ed6887327dabed7aa7da29d7eb0d", GitTreeState:"clean"}
Server: &version.Version{SemVer:"v2.16.3", GitCommit:"1ee0254c86d4ed6887327dabed7aa7da29d7eb0d", GitTreeState:"clean"}


In [14]:
!helm repo add dask-gateway https://dask.org/dask-gateway-helm-repo

"dask-gateway" has been added to your repositories


In [15]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Skip local chart repository
...Successfully got an update from the "dask-gateway" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete.


In [45]:
!openssl rand -hex 32

3beeb083391169fc6cfa2762862d0c1d05638ed7baaf9df9aa546e85731fc730


In [46]:
%%writefile config.yaml
gateway:
    proxyToken: "3beeb083391169fc6cfa2762862d0c1d05638ed7baaf9df9aa546e85731fc730"

Writing config.yaml


In [51]:
cmd = """RELEASE=dask-gateway
NAMESPACE=dask-gateway
VERSION=0.6.1

helm upgrade --install \
    --namespace $NAMESPACE \
    --version $VERSION \
    --values config.yaml \
    $RELEASE \
    dask-gateway/dask-gateway
"""

cmd

'RELEASE=dask-gateway\nNAMESPACE=dask-gateway\nVERSION=0.6.1\n\nhelm upgrade --install     --namespace $NAMESPACE     --version $VERSION     --values config.yaml     $RELEASE     dask-gateway/dask-gateway\n'

In [52]:
os.system(cmd)

0

In [56]:
!kubectl get service --namespace dask-gateway

NAME                            TYPE           CLUSTER-IP     EXTERNAL-IP      PORT(S)          AGE
gateway-api-dask-gateway        ClusterIP      10.0.75.163    <none>           8001/TCP         2m35s
scheduler-api-dask-gateway      ClusterIP      10.0.240.33    <none>           8001/TCP         2m35s
scheduler-public-dask-gateway   LoadBalancer   10.0.180.252   52.156.150.215   8786:32738/TCP   2m35s
web-api-dask-gateway            ClusterIP      10.0.31.94     <none>           8001/TCP         2m35s
web-public-dask-gateway         LoadBalancer   10.0.209.90    40.91.78.219     80:31338/TCP     2m35s


In [55]:
from dask_gateway import Gateway

In [57]:
gateway = Gateway('http://40.91.78.219', proxy_address='tls://52.156.150.215:8786')

In [58]:
gateway.list_clusters()

[]

In [60]:
cluster = gateway.new_cluster()

In [92]:
cluster

In [93]:
c = cluster.get_client()
c

OSError: Timed out trying to connect to 'gateway://52.156.150.215:8786/48670c72b0b74d958b452a50c10dffad' after 10 s: Timed out trying to connect to 'gateway://52.156.150.215:8786/48670c72b0b74d958b452a50c10dffad' after 10 s: connect() didn't finish in time

In [86]:
options = gateway.cluster_options()

In [87]:
import dask

In [88]:
@dask.delayed
def testing():
    return 'hi'

In [89]:
testing().compute()

'hi'

In [90]:
cluster.shutdown()

In [ ]:
!az group delete -n $RG --no-wait -y

In [59]:
from dask_kubernetes import KubeCluster, make_pod_spec

In [ ]:
pod_spec = make_pod_spec(image='daskdev/dask:latest',
                          memory_limit='1G', memory_request='1G',
                          cpu_limit=1, cpu_request=1,
                          env={'EXTRA_PIP_PACKAGES': 'dask-lightgbm'}
                        ) 

In [ ]:
cluster = KubeCluster(pod_spec)

In [ ]:
cluster

In [ ]:
cluster.scale(10)

In [ ]:
from dask.distributed import Client

In [ ]:
c = Client(cluster)

In [ ]:
c

In [ ]:
c.get_versions(check=True)

In [ ]:
import dask

In [ ]:
@dask.delayed
def testing():
    print('hi')

In [ ]:
testing().compute()

In [ ]:
cluster.close()